### Multilayer Neural Network with Tf's Keras API

In [2]:
import os
import struct
import numpy as np

In [3]:
def load_mnist(path, kind='train'):
    """Load MNIST data from path"""
    labels_path=os.path.join(path, '%s-labels.idx1-ubyte' % kind)
    images_path=os.path.join(path, '%s-images.idx3-ubyte' % kind)
    
    with open(labels_path, 'rb') as lbpath:
        magic, n=struct.unpack('>II', lbpath.read(8))
        labels=np.fromfile(lbpath, dtype=np.uint8)
    
    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols=struct.unpack('>IIII', imgpath.read(16))
        images=np.fromfile(imgpath, dtype=np.uint8).reshape(len(labels), 784)
        images=((images/255.0)-0.5)*2 #normalized to -1 to 1 range
    
    return images, labels

In [4]:
# Loading the data
X_train, y_train=load_mnist('data/',kind='train')
print('Rows: %d, Columns: %d'%(X_train.shape[0], X_train.shape[1]))

Rows: 60000, Columns: 784


In [5]:
X_test, y_test=load_mnist('data/', kind='t10k')
print('Rows: %d, Columns: %d'%(X_test.shape[0], X_test.shape[1]))

Rows: 10000, Columns: 784


In [6]:
# Mean centering and normalization
mean_vals=np.mean(X_train, axis=0)
std_val=np.std(X_train)
X_train_centered=(X_train-mean_vals)/std_val
X_test_centered=(X_test-mean_vals)/std_val

In [7]:
# Setting random seed for numpy and tensorflow
import tensorflow as tf
import tensorflow.contrib.keras as keras
np.random.seed(123)
tf.set_random_seed(123)

In [8]:
# Conveting class labels into one-hot format
y_train_onehot=keras.utils.to_categorical(y_train)
print('First 3 labels:',y_train[:3])
print('First 3 labels(one-hot):\n',y_train_onehot[:3])

First 3 labels: [5 0 4]
First 3 labels(one-hot):
 [[ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]]


In [9]:
# Implementing the neural network
model=keras.models.Sequential()
model.add(
    keras.layers.Dense(
        units=50,
        input_dim=X_train_centered.shape[1],
        kernel_initializer='glorot_uniform',
        bias_initializer='zeros',
        activation='tanh'))
model.add(
    keras.layers.Dense(
        units=50,
        input_dim=50,
        kernel_initializer='glorot_uniform',
        bias_initializer='zeros',
        activation='tanh'))
model.add(
    keras.layers.Dense(
        units=y_train_onehot.shape[1],
        input_dim=50,
        kernel_initializer='glorot_uniform',
        bias_initializer='zeros',
        activation='softmax'))
sgd_optimizer=keras.optimizers.SGD(lr=0.001, decay=1e-7, momentum=0.9)
model.compile(optimizer=sgd_optimizer, loss='categorical_crossentropy')

In [10]:
history=model.fit(X_train_centered, y_train_onehot, batch_size=64, epochs=50, verbose=1, validation_split=0.1)

Train on 54000 samples, validate on 6000 samples
Epoch 1/50
54000/54000 [==============================] - 38s 700us/step - loss: 0.7247 - val_loss: 0.3616
Epoch 2/50
54000/54000 [==============================] - 3s 56us/step - loss: 0.3718 - val_loss: 0.2815
Epoch 3/50
54000/54000 [==============================] - 3s 59us/step - loss: 0.3087 - val_loss: 0.2447
Epoch 4/50
54000/54000 [==============================] - 3s 54us/step - loss: 0.2728 - val_loss: 0.2216
Epoch 5/50
54000/54000 [==============================] - 3s 60us/step - loss: 0.2475 - val_loss: 0.2042
Epoch 6/50
54000/54000 [==============================] - 3s 61us/step - loss: 0.2277 - val_loss: 0.1918
Epoch 7/50
54000/54000 [==============================] - 4s 69us/step - loss: 0.2115 - val_loss: 0.1810
Epoch 8/50
54000/54000 [==============================] - 3s 60us/step - loss: 0.1979 - val_loss: 0.1719
Epoch 9/50
54000/54000 [==============================] - 3s 54us/step - loss: 0.1860 - val_loss: 0.1646
Epoc

In [11]:
# Printing model accuracy on training and test sets
y_train_pred=model.predict_classes(X_train_centered,verbose=0)
correct_preds=np.sum(y_train==y_train_pred, axis=0)
train_acc=correct_preds/y_train.shape[0]
print('Training Accuracy: %.2f%%'%(train_acc*100))

y_test_pred=model.predict_classes(X_test_centered, verbose=0)
correct_preds=np.sum(y_test==y_test_pred, axis=0)
test_acc=correct_preds/y_test.shape[0]
print('Test Accuracy: %.2f%%'%(test_acc*100))

Training Accuracy: 98.88%
Test Accuracy: 96.04%
